In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
from sklearn import linear_model
from sklearn import tree

In [2]:
df_team_stats = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_team_stats_final.csv")
df_players_stats = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_players_stats_final.csv")
df_Games = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_final.csv")

In [ ]:
df_team_stats.drop('Unnamed: 0',axis=1,inplace=True)
df_players_stats.drop('Unnamed: 0',axis=1,inplace=True)
df_Games.drop('Unnamed: 0',axis=1,inplace=True)
df_stats = df_players_stats

In [ ]:
# Lista de games e das medias

lista_games_2024 = df_stats['fixture_id'].unique().tolist()
List_AVG = ['rating','minutes','offsides','shots_total','shots_on','goals_total','goals_conceded','assists','saves','passes_total','passes_key','passes_accuracy','tackles_total','tackles_blocks','tackles_interceptions','duels_total','duels_won','dribbles_attempts','dribbles_success','dribbles_past','fouls_drawn','fouls_committed','cards_yellow','cards_red','penalty_won','penalty_committed','penalty_scored','penalty_missed','penalty_saved']

In [10]:
def calcular_media_ultimos_tres(df, fixture_x):
    # Lista de jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os três últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(5)           #Quantos jogos vou pegar 
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby(['player_id','team_id'])[List_AVG].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left')
    
    return resultado_final

In [62]:
#Sugestão da ia
def get_last_three_avg_stats(df_stats, fixture_id, stat_cols=['rating','shots_total', 'goals_total', 'passes_total','minutes']):
    # Seleciona os jogadores que participaram do fixture_id informado
    jogadores = df_stats.loc[df_stats['fixture_id'] == fixture_id, 'player_id'].unique()
    
    # Filtra os jogos anteriores ao fixture_id para os jogadores selecionados
    df_prev = df_stats[(df_stats['fixture_id'] < fixture_id) & (df_stats['player_id'].isin(jogadores))]
    
    # Para cada jogador, ordena os jogos de forma decrescente (últimos primeiro)
    # e pega os três primeiros, calculando a média das estatísticas desejadas
    resultado = df_prev.groupby('player_id').apply(
        lambda x: x.sort_values('fixture_id', ascending=False).head(3)[stat_cols].mean()
    )
    
    # Converte o resultado para um DataFrame e reseta o índice
    resultado = resultado.reset_index()
    return resultado

# Exemplo de uso:
fixture_id_x = 1180585  # exemplo de fixture_id
df_medias = get_last_three_avg_stats(df_stats, fixture_id_x)

C:\Users\paulo\AppData\Local\Temp\ipykernel_10020\1339446783.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df_prev.groupby('player_id').apply(


In [10]:
#DeepSeek (backup)
def calcular_media_ultimos_tres(df, fixture_x):
    # Lista de jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os três últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(3)
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby('player_id')[
        ['rating', 'shots_total', 'goals_total', 'passes_total', 'minutes']
    ].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left')
    
    return resultado_final

#media_ultimos_jogos = calcular_media_ultimos_tres(df_stats, 1180585)
#media_ultimos_jogos

In [11]:
def calcular_media_ultimos_tres(df, fixture_x):
    # Lista de jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os três últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(5)           #Quantos jogos vou pegar 
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby('player_id')[List_AVG].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left')
    
    return resultado_final

media_ultimos_jogos = calcular_media_ultimos_tres(df_stats, 1180585)

In [11]:
# Lista para armazenar os DataFrames de cada fixture
lista_dfs = []

# Iterar sobre cada fixture_id na lista_games_2024
for fixture_x in lista_games_2024:
    # Calcular a média dos últimos três jogos para o fixture atual
    df_medias = calcular_media_ultimos_tres(df_stats, fixture_x)
    
    # Adicionar a coluna 'fixture_id' ao DataFrame resultante (para identificar o jogo)
    df_medias['fixture_id'] = fixture_x
    
    # Adicionar à lista
    lista_dfs.append(df_medias)

# Concatenar todos os DataFrames da lista em um único df_features
df_features = pd.concat(lista_dfs, ignore_index=True)

In [12]:
df_features = df_features[df_features['fixture_id'] > 1180366]
df_features.fillna(0,inplace=True)
df_features

,player_id,team_id,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,fixture_id
459,10298,118.0,7.30,90.000000,0.0,0.0,0.0,0.0,2.0,0.0,7.0,32.000000,0.0,22.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180367
460,30,118.0,6.90,90.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.000000,2.0,43.0,2.000000,2.0,0.0,12.000000,5.0,1.000000,0.000000,1.000000,2.0,2.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180367
461,353728,118.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180367
462,10377,118.0,6.30,90.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.000000,0.0,40.0,1.000000,0.0,0.0,8.000000,3.0,0.000000,0.000000,0.000000,2.0,2.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180367
463,197383,118.0,6.30,90.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.000000,1.0,40.0,0.000000,0.0,4.0,11.000000,2.0,3.000000,0.000000,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17370,403306,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180733
17371,44436,133.0,1.06,0.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,0.8,0.000000,0.0,0.0,0.400000,0.2,0.200000,0.200000,0.000000,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,1180733
17372,2563,133.0,1.30,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,1.8,0.000000,0.0,0.2,0.400000,0.2,0.200000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180733
17373,454139,133.0,2.10,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,2.666667,1.0,0.666667,0.333333,0.333333,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1180733


In [13]:
df = df_features[df_features['minutes'] >= 45 ]

In [14]:
df_Games['Total_Gols'] = df_Games['Gols_Home'] + df_Games['Gols_Away']
df_Games

,id_partida,id_league,referee,date,Stadium,Home_Team,home.id,Away_Team,away.id,home.winner,Gols_Home,Gols_Away,Half_Time_Gols_Home,Half_Time_Gols_Away,Empate,Total_Gols
0,1180355,71,Rodrigo José Pereira de Lima,2024-04-13T21:30:00+00:00,Estádio José Pinheiro Borda,Internacional,119,Bahia,118,True,2,1,0,0,False,3
1,1180364,71,Bruno Pereira Vasconcelos,2024-04-13T21:30:00+00:00,Estádio Heriberto Hülse,Criciuma,140,Juventude,152,False,1,1,1,0,True,2
2,1180358,71,Maguielson Lima Barbosa,2024-04-14T00:00:00+00:00,Estadio Jornalista Mário Filho,Fluminense,124,RB Bragantino,794,False,2,2,1,0,True,4
3,1180361,71,Alex Gomes,2024-04-14T00:00:00+00:00,MorumBIS,Sao Paulo,126,Fortaleza EC,154,False,1,2,0,0,False,3
4,1180360,71,Yuri Elino Ferreira da Cruz,2024-04-14T19:00:00+00:00,Neo Química Arena,Corinthians,131,Atletico-MG,1062,False,0,0,0,0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1180734,71,Edina Alves Batista,2024-12-08T19:00:00+00:00,Estádio Alfredo Jaconi,Juventude,152,Cruzeiro,135,False,0,1,0,1,False,1
376,1180732,71,Matheus Delgado Candançan,2024-12-08T19:00:00+00:00,Estádio Governador Plácido Aderaldo Castelo,Fortaleza EC,154,Internacional,119,True,3,0,2,0,False,3
377,1180731,71,Sávio Pereira,2024-12-08T19:00:00+00:00,Estádio Nabi Abi Chedid,RB Bragantino,794,Criciuma,140,True,5,1,2,1,False,6
378,1180726,71,Rafael Rodrigo Klein,2024-12-08T19:00:00+00:00,Arena MRV,Atletico-MG,1062,Atletico Paranaense,134,True,1,0,0,0,False,1


In [16]:
df = df.groupby(['fixture_id','team_id'])[List_AVG].mean()

In [17]:
df = df.merge(df_Games[['id_partida', 'Total_Gols']], 
              left_on='fixture_id', right_on='id_partida', how='left')

In [19]:
df

,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,id_partida,Total_Gols
0,6.650000,78.666667,0.083333,0.416667,0.250000,0.000000,0.166667,0.083333,0.583333,37.250000,0.416667,31.083333,0.833333,0.166667,0.750000,9.166667,3.500000,1.416667,0.666667,0.583333,1.333333,0.916667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180367,3
1,7.333333,85.222222,0.000000,1.777778,1.111111,0.222222,0.222222,0.222222,0.666667,44.777778,1.666667,39.333333,0.333333,0.000000,0.444444,8.666667,4.111111,2.444444,0.888889,0.444444,1.666667,0.555556,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180367,3
2,7.045455,80.090909,0.181818,0.636364,0.272727,0.000000,0.000000,0.000000,0.000000,56.727273,0.636364,50.363636,1.181818,0.090909,0.818182,8.818182,5.454545,1.636364,1.090909,0.636364,1.090909,0.818182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180368,3
3,7.036364,80.181818,0.090909,0.818182,0.181818,0.181818,0.090909,0.000000,0.272727,36.818182,0.818182,31.636364,0.545455,0.181818,0.454545,6.727273,3.181818,0.909091,0.545455,0.727273,1.000000,1.000000,0.272727,0.000000,0.090909,0.090909,0.090909,0.000000,0.0,1180368,3
4,6.609091,75.909091,0.000000,0.454545,0.181818,0.090909,0.000000,0.090909,0.181818,22.818182,0.454545,17.909091,0.909091,0.454545,0.636364,6.909091,3.000000,1.090909,0.727273,0.636364,0.636364,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180369,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,6.842000,78.040000,0.040000,0.540000,0.280000,0.080000,0.140000,0.080000,0.420000,24.860000,0.540000,20.420000,1.200000,0.260000,0.740000,6.440000,3.280000,0.860000,0.500000,0.580000,0.880000,0.920000,0.120000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,1180732,3
771,6.735556,73.622222,0.155556,0.555556,0.200000,0.066667,0.200000,0.066667,0.511111,32.800000,0.400000,28.422222,0.866667,0.266667,0.600000,5.177778,2.755556,0.511111,0.311111,0.288889,0.822222,0.755556,0.177778,0.022222,0.000000,0.000000,0.000000,0.000000,0.0,1180733,3
772,6.863333,71.600000,0.033333,0.366667,0.200000,0.066667,0.000000,0.033333,0.000000,20.566667,0.366667,16.233333,1.433333,0.400000,0.500000,7.066667,3.700000,1.133333,0.533333,0.466667,0.800000,0.766667,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180733,3
773,6.746667,79.111111,0.088889,0.933333,0.288889,0.088889,0.133333,0.044444,0.444444,40.155556,0.955556,35.155556,0.844444,0.200000,0.288889,6.333333,2.822222,1.200000,0.488889,0.577778,0.733333,0.911111,0.155556,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180734,1


In [31]:
df['Total_Gols'] = df['Total_Gols'] > 1.5

In [32]:
df['Total_Gols'] = df['Total_Gols'].astype(int)

In [33]:
df

,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,id_partida,Total_Gols
0,6.942857,81.476190,0.047619,1.000000,0.619048,0.095238,0.190476,0.142857,0.619048,40.476190,0.952381,34.619048,0.619048,0.095238,0.619048,8.952381,3.761905,1.857143,0.761905,0.523810,1.476190,0.761905,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180367,1
1,7.040909,80.136364,0.136364,0.727273,0.227273,0.090909,0.045455,0.000000,0.136364,46.772727,0.727273,41.000000,0.863636,0.136364,0.636364,7.772727,4.318182,1.272727,0.818182,0.681818,1.045455,0.909091,0.136364,0.000000,0.045455,0.045455,0.045455,0.000000,0.0,1180368,1
2,6.680952,73.761905,0.095238,0.571429,0.285714,0.095238,0.095238,0.047619,0.095238,19.761905,0.571429,14.904762,0.714286,0.333333,0.428571,6.857143,3.238095,1.095238,0.714286,0.619048,0.904762,0.904762,0.142857,0.000000,0.047619,0.000000,0.000000,0.047619,0.0,1180369,0
3,7.108333,73.500000,0.208333,0.958333,0.500000,0.083333,0.041667,0.041667,0.333333,28.208333,0.708333,23.208333,2.000000,0.250000,0.708333,8.625000,4.875000,0.666667,0.458333,0.750000,1.000000,1.250000,0.291667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180370,0
4,7.112500,74.000000,0.041667,0.833333,0.458333,0.166667,0.041667,0.125000,0.250000,23.708333,0.791667,18.916667,1.833333,0.250000,0.416667,8.500000,4.500000,1.125000,0.791667,0.750000,0.666667,1.416667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1180371,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,6.832500,72.258333,0.150000,0.908333,0.408333,0.100000,0.083333,0.041667,0.216667,34.733333,1.075000,29.733333,1.091667,0.150000,0.583333,6.941667,3.450000,1.350000,0.691667,0.516667,0.758333,0.758333,0.141667,0.016667,0.008333,0.008333,0.000000,0.016667,0.0,1180730,0
363,6.663158,76.915789,0.084211,0.747368,0.273684,0.073684,0.168421,0.073684,0.378947,27.200000,0.810526,21.768421,1.273684,0.357895,0.663158,7.463158,3.894737,1.157895,0.610526,0.778947,0.863158,0.915789,0.157895,0.000000,0.000000,0.021053,0.000000,0.000000,0.0,1180731,1
364,6.891000,74.170000,0.100000,0.630000,0.330000,0.080000,0.120000,0.090000,0.300000,27.930000,0.510000,22.930000,1.240000,0.170000,0.800000,6.620000,3.280000,1.020000,0.520000,0.570000,0.820000,0.880000,0.150000,0.000000,0.010000,0.000000,0.000000,0.010000,0.0,1180732,1
365,6.786667,72.813333,0.106667,0.480000,0.200000,0.066667,0.120000,0.053333,0.306667,27.906667,0.386667,23.546667,1.093333,0.320000,0.560000,5.933333,3.133333,0.760000,0.400000,0.360000,0.813333,0.760000,0.186667,0.013333,0.000000,0.000000,0.000000,0.000000,0.0,1180733,1


# Modelo de Test

In [34]:
features = df.columns[0:-2].tolist()
target = 'Total_Gols'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(df[features],df[target], test_size= 0.2, random_state=42, stratify=df[target]) 

In [35]:
modelo = ensemble.RandomForestClassifier()
modelo.fit(X_train, y_train)


#---
y_train_pred = modelo.predict(X_train)
y_train_prob = modelo.predict_proba(X_train)
y_test_pred = modelo.predict(X_test)
y_test_prob = modelo.predict_proba(X_test)
print(f'Acurácia treino: {metrics.accuracy_score(y_train, y_train_pred):.4f}')
print(f'Acurácia teste: {metrics.accuracy_score(y_test, y_test_pred):.4f}')
print(f'AUC treino: {metrics.roc_auc_score(y_train, y_train_prob[:,1]):.4f}')
print(f'AUC teste: {metrics.roc_auc_score(y_test, y_test_prob[:,1]):.4f}')

print(f'Proporção de target=1 (treino): {y_train.mean():.2%}')
print(f'Proporção de target=1 (teste): {y_test.mean():.2%}')

Acurácia treino: 1.0000
Acurácia teste: 0.7162
AUC treino: 1.0000
AUC teste: 0.5903
Proporção de target=1 (treino): 72.35%
Proporção de target=1 (teste): 72.97%


In [74]:
y_train_pred = modelo.predict(X_train)
y_train_prob = modelo.predict_proba(X_train)
y_test_pred = modelo.predict(X_test)
y_test_prob = modelo.predict_proba(X_test)

In [67]:
print(f'Acurácia treino: {metrics.accuracy_score(y_train, y_train_pred):.4f}')
print(f'Acurácia teste: {metrics.accuracy_score(y_test, y_test_pred):.4f}')
print(f'AUC treino: {metrics.roc_auc_score(y_train, y_train_prob[:,1]):.4f}')
print(f'AUC teste: {metrics.roc_auc_score(y_test, y_test_prob[:,1]):.4f}')

print(f'Proporção de target=1 (treino): {y_train.mean():.2%}')
print(f'Proporção de target=1 (teste): {y_test.mean():.2%}')

Acurácia treino: 0.9863
Acurácia teste: 0.5946
AUC treino: 0.9988
AUC teste: 0.4366
Proporção de target=1 (treino): 72.35%
Proporção de target=1 (teste): 72.97%
